# Download and convert to Geopackage

This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Geopackage Database for future use.
Geopackage is a convenient data storage format, based on SQL, and is compatible with QGIS.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon (could have been set)
# gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')
gdf_geom = gpd.read_file('/home/vschaffn/Documents/swot_data/aoi.gpkg')

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,7),
)


## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [ ]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a geopackage database.
This geopackage format is quite efficient (though not the most efficient), and may easily be visualized in, e.g., QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.gpkg import Nc2GpkgConverter
from glob import glob

In [5]:
pixc = Nc2GpkgConverter(
            glob(pixcdownloader.path_download+'/*/*nc'),
            "/tmp/pixc_gpkg.gpkg",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

 50%|████████████████████████████████████████████████████████████████████████████████████████████                                                                                            | 1/2 [00:05<00:05,  5.57s/it]

--File /tmp/pixc/URN_FEATURE_DATA_swot_cb8c5eee-ae7e-34b8-baa5-482a8d220531_V1-URN_FEATURE_DATA_swot_cb8c5eee-ae7e-34b8-baa5-482a8d220531_V1/SWOT_L2_HR_PIXC_482_016_077L_20230406T094608_20230406T094619_PGC0_01.nc combined with area of interest                        returned empty. Skipping it


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [04:37<00:00, 138.53s/it]


database has been succesfully created, we can remove the raw files

In [16]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
previous steps are not necessary

Now we can open this database in a GeoDataFrame, load it in, e.g., QGIS, etc.

In [6]:
from pixcdust.readers.gpkg import GpkgReader

# nb: you may specify 
pixc_read = PixCGpkgReader(
    "/tmp/pixc_gpkg.gpkg"
)
pixc_read.read()
pixc_read.data

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.98s/it]


,height,sig0,classification,geoid,latitude,longitude,wse,geometry
0,254.042862,6.551862,6.0,49.569977,43.441763,1.222368,204.472885,POINT (1.22237 43.44176)
1,247.325134,0.184823,1.0,49.569973,43.441765,1.222379,197.755157,POINT (1.22238 43.44177)
2,250.669281,0.566375,2.0,49.569908,43.441785,1.222488,201.099365,POINT (1.22249 43.44178)
3,248.438095,0.689724,1.0,49.569904,43.441786,1.222494,198.868195,POINT (1.22249 43.44179)
4,254.293930,11.020334,6.0,49.569794,43.441820,1.222685,204.724136,POINT (1.22268 43.44182)
...,...,...,...,...,...,...,...,...
2827186,271.792694,-1.606044,1.0,49.451050,43.735045,1.574508,222.341644,POINT (1.57451 43.73504)
2827187,271.873932,-1.053646,1.0,49.450874,43.735218,1.574340,222.423065,POINT (1.57434 43.73522)
2827188,269.939667,0.699450,1.0,49.450420,43.735340,1.573886,220.489243,POINT (1.57389 43.73534)
2827189,273.954163,2.845398,1.0,49.450687,43.735594,1.574173,224.503479,POINT (1.57417 43.73559)


Enjoy!